In [1]:
import datetime
import dask.dataframe as dd
import pandas as pd
import numpy as np
from featurelib import Engine
from solution import FavouriteStoreCalcer

In [2]:
purchases_dd = dd.read_parquet('data/purchases.parquet')
purchases_dd.head()

,client_id,transaction_id,transaction_datetime,regular_points_received,express_points_received,regular_points_spent,express_points_spent,purchase_sum,store_id,product_id,product_quantity,trn_sum_from_iss,trn_sum_from_red
0,170057,9797969,2019-01-29 15:28:44,6.0,0.0,0.0,0.0,608.68,13327,10810,1.0,5.0,NaN
1,170057,9797969,2019-01-29 15:28:44,6.0,0.0,0.0,0.0,608.68,13327,1358,1.0,170.0,NaN
2,170057,9797969,2019-01-29 15:28:44,6.0,0.0,0.0,0.0,608.68,13327,17306,1.0,93.0,NaN
3,170057,9329647,2019-01-24 05:25:02,0.6,0.0,0.0,0.0,138.31,13327,37296,1.0,88.0,NaN
4,170057,9797969,2019-01-29 15:28:44,6.0,0.0,0.0,0.0,608.68,13327,30142,1.0,147.0,NaN


In [3]:
receipts_dd = dd.read_parquet('data/receipts.parquet')
receipts_dd.head()

,client_id,transaction_id,trn_sum_from_red,trn_sum_from_iss,regular_points_received,express_points_received,regular_points_spent,express_points_spent,purchase_sum,store_id,transaction_datetime
0,5368,572976,0.0,427.0,2.1,0.0,0.0,0.0,427.00,2762,2018-11-27 15:52:50
1,5368,1095593,0.0,438.0,2.1,0.0,0.0,0.0,437.00,3395,2019-02-23 10:22:23
2,5368,1843577,0.0,1890.0,14.7,0.0,0.0,0.0,1889.00,3395,2019-02-23 10:21:29
3,5368,2389159,0.0,3274.0,32.7,0.0,0.0,0.0,3273.00,3395,2018-12-15 10:41:28
4,5368,3766070,0.0,1585.0,11.7,0.0,0.0,0.0,1585.43,3395,2019-02-08 08:41:18


In [4]:
engine = Engine(tables={})
engine.register_table(table=dd.read_parquet('data/purchases.parquet'),
                      name='purchases')
engine.register_table(table=dd.read_parquet('data/receipts.parquet'),
                      name='receipts')
engine.register_table(table=dd.read_csv('data/products.csv'),
                      name='products')
engine.register_table(table=dd.read_csv('data/client_profile.csv'),
                      name='client_profile')
engine.register_table(table=dd.read_csv('data/campaigns.csv'),
                      name='campaigns')

test_df = pd.read_csv('dataset_mini.csv')

calcer = FavouriteStoreCalcer(engine, **{'delta': 30, 'date_to': datetime.date(2019, 3, 19)})
calcer

In [5]:
result_df = calcer.compute().compute()
result_df.head()

,client_id,favourite_store_id__30d
0,5368,3395
1,11882,125
2,15640,11168
3,17317,12767
4,25169,2200


In [6]:
mask = result_df[result_df.columns[-1]] != test_df[result_df.columns[-1]]
result_df[mask]

,client_id,favourite_store_id__30d
79,410135,9298
81,413670,2585
95,470521,12714


In [7]:
test_df[mask][result_df.columns]

,client_id,favourite_store_id__30d
79,410135,13044
81,413670,13159
95,470521,12800


In [9]:
source_dd = receipts_dd.copy()
date_to = pd.to_datetime(datetime.date(2019, 3, 19))
date_from = pd.to_datetime(datetime.date(2019, 3, 19) - datetime.timedelta(days=30))

mask = ((source_dd['transaction_datetime'] < date_to)
        & (source_dd['transaction_datetime'] >= date_from))
source_dd[mask & (source_dd['client_id'] == 410135)].groupby('store_id')['purchase_sum'].agg(['sum', 'count']).compute()

,sum,count
store_id,,
9298,420.94,3
10165,135.29,1
12720,674.89,2
13044,295.23,3
